In [ ]:
import geopandas as gpd # For vector data processing
import folium # For interactive web maps

ID_FEMA_lidar_tile_fname = 'ID_FEMAHQ_2018_las_tiles.geojson'
ubrb_outline_fname = '../data/ubrb/upper_boise_geometry.shp'

In [ ]:
lidar_tile_gdf = gpd.read_file(ID_FEMA_lidar_tile_fname)
lidar_tile_gdf.head()

In [ ]:
ubrb_watershed_gdf = gpd.read_file(ubrb_outline_fname)
ubrb_watershed_gdf.head()

In [ ]:
centerLat = (lidar_tile_gdf.total_bounds[1] + lidar_tile_gdf.total_bounds[3])/2
centerLon = (lidar_tile_gdf.total_bounds[0] + lidar_tile_gdf.total_bounds[2])/2

In [ ]:
m = folium.Map(location=[centerLat, centerLon], zoom_start=9) # Create folium map centered on the riparian LULC raster

# Replace with your desired satellite tile URL
satellite_url = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'

# Let's add a satellite data layer so we can see the riparian buffer LULC in a visual context
folium.TileLayer(
    tiles=satellite_url,
    attr='Esri Satellite', # Add attribution here
    name='Esri Satellite',
    opacity=0.6
).add_to(m)

# Add ubrb watershed GeoDataFrame as a GeoJson overlay
folium.GeoJson(
    ubrb_watershed_gdf,
    style_function=lambda feature: { # Lambda is like an in-line for loop
        'fillColor': 'lightblue',
        'color': 'darkblue',
        'weight': 1,
        'fillOpacity': 0.1}).add_to(m)

# Create a tooltip for mouse hover over
tooltip = folium.features.GeoJsonTooltip(
    fields=['tile_id', 'browse_url', 'laz_url'],
    localize=True,
    sticky=False,
    labels=True,
    style='''
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    ''',
    max_width=800,
)

# Add lidar tile GeoDataFrame as a GeoJson overlay
folium.GeoJson(
    lidar_tile_gdf,
    tooltip=tooltip,
    style_function=lambda feature: { # Lambda is like an in-line for loop
        'fillColor': 'green',
        'color': 'green',
        'weight': 0.5,
        'fillOpacity': 0.2}).add_to(m)

folium.LayerControl().add_to(m) # Add layer control to toggle layers on and off

m # Display the map